In [ ]:
import os
import numpy as np
import cv2
import json

## Imports for plotting
import matplotlib.pyplot as plt
from PIL import Image
from numpy import asarray
from IPython.display import clear_output

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

# Torchvision
import torchvision
from torchvision import transforms


DATASET_PATH = "../data"
CHECKPOINT_PATH = "../saved_models/tutorial10"

device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

In [ ]:
import urllib.request
from urllib.error import HTTPError
import zipfile

# Files to download
pretrained_files = [(DATASET_PATH, "TinyImageNet.zip")]
# Create checkpoint path if it doesn't exist yet
os.makedirs(DATASET_PATH, exist_ok=True)
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial10/"

# For each file, check whether it already exists. If not, try downloading it.
for dir_name, file_name in pretrained_files:
    file_path = os.path.join(dir_name, file_name)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print(f"Downloading {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)
        if file_name.endswith(".zip"):
            print("Unzipping file...")
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(file_path.rsplit("/",1)[0])

In [ ]:
# Load CNN architecture pretrained on ImageNet
os.environ["TORCH_HOME"] = CHECKPOINT_PATH
pretrained_model = torchvision.models.resnet34(pretrained=True)
pretrained_model = pretrained_model.to(device)

# No gradients needed for the network
pretrained_model.eval()
for p in pretrained_model.parameters():
    p.requires_grad = False

In [ ]:
# Mean and Std from ImageNet
NORM_MEAN = np.array([0.485, 0.456, 0.406])
NORM_STD = np.array([0.229, 0.224, 0.225])
# No resizing and center crop necessary as images are already preprocessed.
plain_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN,std=NORM_STD),
    # transforms.Resize((256, 256))
])

# Load dataset and create data loader
imagenet_path = os.path.join(DATASET_PATH, "TinyImageNet/")
assert os.path.isdir(imagenet_path), f"Could not find the ImageNet dataset at expected path \"{imagenet_path}\". " + \
                                     f"Please make sure to have downloaded the ImageNet dataset here, or change the {DATASET_PATH} variable."
dataset = torchvision.datasets.ImageFolder(root=imagenet_path, transform=plain_transforms)
data_loader = data.DataLoader(dataset, batch_size=32, shuffle=False, drop_last=False, num_workers=8)

# Load label names to interpret the label numbers 0 to 999
with open(os.path.join(imagenet_path, "label_list.json"), "r") as f:
    label_names = json.load(f)

def get_label_index(lab_str):
    assert lab_str in label_names, f"Label \"{lab_str}\" not found. Check the spelling of the class."
    return label_names.index(lab_str)

In [ ]:
def eval_model(dataset_loader, img_func=None):
    tp, tp_5, counter = 0., 0., 0.
    for imgs, labels in tqdm(dataset_loader, desc="Validating..."):
        imgs = imgs.to(device)
        labels = labels.to(device)
        if img_func is not None:
            imgs = img_func(imgs, labels)
        with torch.no_grad():
            preds = pretrained_model(imgs)
        tp += (preds.argmax(dim=-1) == labels).sum()
        tp_5 += (preds.topk(5, dim=-1)[1] == labels[...,None]).any(dim=-1).sum()
        counter += preds.shape[0]
    acc = tp.float().item()/counter
    top5 = tp_5.float().item()/counter
    print(f"Top-1 error: {(100.0 * (1 - acc)):4.2f}%")
    print(f"Top-5 error: {(100.0 * (1 - top5)):4.2f}%")
    return acc, top5

In [ ]:
def place_patch(img, patch):
    for i in range(img.shape[0]):
        h_offset = np.random.randint(0,img.shape[2]-patch.shape[1]-1)
        w_offset = np.random.randint(0,img.shape[3]-patch.shape[2]-1)
        img[i,:,h_offset:h_offset+patch.shape[1],w_offset:w_offset+patch.shape[2]] = patch_forward(patch)
    return img

In [ ]:
TENSOR_MEANS, TENSOR_STD = torch.FloatTensor(NORM_MEAN)[:,None,None], torch.FloatTensor(NORM_STD)[:,None,None]
def patch_forward(patch):
    # Map patch values from [-infty,infty] to ImageNet min and max
    patch = (torch.tanh(patch) + 1 - 2 * TENSOR_MEANS) / (2 * TENSOR_STD)
    return patch

In [ ]:
def eval_patch(model, patch, val_loader, target_class):
    model.eval()
    tp, tp_5, counter = 0., 0., 0.
    with torch.no_grad():
        for img, img_labels in tqdm(val_loader, desc="Validating...", leave=False):
            # For stability, place the patch at 4 random locations per image, and average the performance
            for _ in range(4):
                patch_img = place_patch(img, patch)
                patch_img = patch_img.to(device)
                img_labels = img_labels.to(device)
                pred = model(patch_img)
                # In the accuracy calculation, we need to exclude the images that are of our target class
                # as we would not "fool" the model into predicting those
                tp += torch.logical_and(pred.argmax(dim=-1) == target_class, img_labels != target_class).sum()
                tp_5 += torch.logical_and((pred.topk(5, dim=-1)[1] == target_class).any(dim=-1), img_labels != target_class).sum()
                counter += (img_labels != target_class).sum()
    acc = tp/counter
    top5 = tp_5/counter
    return acc, top5

In [ ]:
def patch_attack(model, target_class, patch_size, num_epochs, pretrained_patch):
    # Leave a small set of images out to check generalization
    # In most of our experiments, the performance on the hold-out data points
    # was as good as on the training set. Overfitting was little possible due
    # to the small size of the patches.
    train_set, val_set = torch.utils.data.random_split(dataset, [4800, 200])
    train_loader = data.DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True, num_workers=8)
    val_loader = data.DataLoader(val_set, batch_size=32, shuffle=False, drop_last=False, num_workers=4)

    # Create parameter and optimizer
    if not isinstance(patch_size, tuple):
        patch_size = (patch_size, patch_size)
    if pretrained_patch is None:
        patch = nn.Parameter(torch.zeros(3, patch_size[0], patch_size[1]), requires_grad=True)
    else:
        patch_np = np.load(pretrained_patch)
        patch = nn.Parameter(torch.tensor(patch_np), requires_grad=True)

    optimizer = torch.optim.SGD([patch], lr=1e-1, momentum=0.5)
    loss_module = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(num_epochs):
        clear_output(wait=True)
        patch_temp=patch.detach().cpu().permute(2, 1, 0)
        patch_temp = (torch.tanh(patch_temp) + 1) / 2
        patch_temp = np.clip(patch_temp, a_min=0.0, a_max=1.0)
        plt.imshow(patch_temp)
        plt.grid(False)
        plt.show()

        t = tqdm(train_loader, leave=False)
        for img, _ in t:
            img = place_patch(img, patch)
            img = img.to(device)
            pred = model(img)
            labels = torch.zeros(img.shape[0], device=pred.device, dtype=torch.long).fill_(target_class)
            loss = loss_module(pred, labels)
            optimizer.zero_grad()
            loss.mean().backward()
            optimizer.step()
            t.set_description(f"Epoch {epoch}, Loss: {loss.item():4.2f}")

    # Final validation
    acc, top5 = eval_patch(model, patch, val_loader, target_class)

    return patch.data, {"acc": acc.item(), "top5": top5.item()}

In [ ]:
patch, metrics = patch_attack(model = pretrained_model, 
                              target_class = 761,
                              patch_size = 64,
                              num_epochs = 32,
                              pretrained_patch=None)

np.save('patch_761.npy', patch)
patch = patch.cpu().permute(2, 1, 0)
patch = (torch.tanh(patch) + 1) / 2
patch = np.clip(patch, a_min=0.0, a_max=1.0)
# save patch

In [ ]:
!wget  "https://upload.wikimedia.org/wikipedia/commons/thumb/8/85/Senior_Airman_Benton_Pohlman_fires_an_M4_carbine_rifle_%2834135723246%29.jpg/1200px-Senior_Airman_Benton_Pohlman_fires_an_M4_carbine_rifle_%2834135723246%29.jpg" -O "pistol.jpg"

In [ ]:
patch=np.load('/content/patch_761.npy').transpose(2,1,0)

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def attack_image(input_normed, input_raw, attack_patch,):
  # input_numpy = asarray(Image.open(image_path))

#   input_numpy = NormalizeData(input_numpy)
  patch_show = (np.tanh(attack_patch) + 1) / 2
#   patch_show = np.clip(patch, a_min=0.0, a_max=1.0)

  h_offset = np.random.randint(0,input_normed.shape[0]-attack_patch.shape[0]-1)
  w_offset = np.random.randint(0,input_normed.shape[1]-attack_patch.shape[1]-1)
  ret = input_normed.copy()
  ret[h_offset:h_offset+patch.shape[0],w_offset:w_offset+patch.shape[1]] = attack_patch

  input_raw = NormalizeData(input_raw)
  input_raw[h_offset:h_offset+patch.shape[0],w_offset:w_offset+patch.shape[1]] = patch_show
  return ret, input_raw

In [ ]:
#plot top 5 inference scores on original and attacked
def indices_to_names(indices, label_names):
  names = label_names[indices]
  return(names)

def top_5_predictions(input_numpy, patch, resize_input, model, label_names, transforms):
  if resize_input:
      input_numpy = cv2.resize(input_numpy, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)

  input_show = np.empty_like(input_numpy)
  input_show[:] = input_numpy

  fig, ax = plt.subplots(nrows=2, ncols=2)
  ax[0][0].imshow(input_show)

  input_numpy = transforms(input_numpy).numpy().transpose(1,2,0)
  attacked_input_numpy, attacked_input_show = attack_image(input_numpy, input_show, patch)
  
  input_1 = torch.tensor(input_numpy.transpose(2,1,0)).unsqueeze(0).type(torch.cuda.FloatTensor).to(device)
  input_2 = torch.tensor(attacked_input_numpy.transpose(2, 1, 0)).unsqueeze(0).type(torch.cuda.FloatTensor).to(device)

  results_1 = torch.softmax(model(input_1), dim=-1).cpu()[0]
  results_2 = torch.softmax(model(input_2), dim=-1).cpu()[0]
  values_1, indices_1 = results_1.topk(5)
  values_2, indices_2 = results_2.topk(5)

  argmax_1, argmax_2 = torch.argmax(results_1), torch.argmax(results_2)
  names_1, names_2 = label_names[indices_1.cpu().numpy()], label_names[indices_2.cpu().numpy()]

  plt.rcParams["figure.figsize"] = (9,9)
  
  ax[0][1].imshow(attacked_input_show)
  ax[1][0].bar([i for i in range(5)], values_1)
  ax[1][0].set_xticks([i for i in range(5)])
  ax[1][0].set_xticklabels(names_1, rotation=40)
  
  ax[1][1].bar([i for i in range(5)], values_2)
  ax[1][1].set_xticks([i for i in range(5)])
  ax[1][1].set_xticklabels(names_2, rotation=40)

In [ ]:
i=plt.imread('pistol.jpg')
top_5_predictions(input_numpy=i, 
                  patch=patch, 
                  resize_input=True,
                  model=pretrained_model, 
                  label_names=np.array(label_names), 
                  transforms=plain_transforms)